In [177]:
#http://127.0.0.1:8050/

In [178]:
# Import
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
import os
import dash_cytoscape as cyto
import json
import igraph as ig
import dash_bootstrap_components as dbc

cyto.load_extra_layouts()

graph_elements = None

#TODO Get from config.josn
output_root_path = "./pdac/output"

In [179]:
def graphml_to_cytoscape(graphml_path):
    #create graph from graphml
    g = ig.Graph.Read_GraphML(graphml_path)
    graph_elements = []
    index_clusters = []
    #convert and add vertex
    for e in g.vs():
        _map = e.attributes()
        _map["id"] = e.index
        graph_elements.append({"data": _map, "classes": e["vertex_type"], "id": e.index, "grabbable": False, })
        index_clusters.append(int(e["cluster"]))
    #convert and add edges
    for e in g.es():
        graph_elements.append({"data": {"source": e.source, "target": e.target}})
    
    #list of all clusters index
    index_clusters = list(set(index_clusters))

    return graph_elements, index_clusters

def filter_elements_with_cluster(cluster):
    global graph_elements
    filtered_graph_elements = []
    list_index_vertex = []
    #filter vertex
    for e in graph_elements:
        if e["data"].get("cluster", -1) == cluster:
            filtered_graph_elements.append(e)
            list_index_vertex.append(e["id"])
    #filter edges
    for e in graph_elements:
        if e["data"].get("source", -1) in list_index_vertex and e["data"].get("target", -1) in list_index_vertex:
            filtered_graph_elements.append(e)

    return filtered_graph_elements

In [180]:


# Data graph all
graphml_path = os.path.join(output_root_path, "grafo_cytoscape.graphml")
graph_elements, clusters = graphml_to_cytoscape(graphml_path)

# I files extra vengono letti e caricati in automatico dalla folder assets (CSS, icon, etc)
app = Dash(title='Tool-Network', external_stylesheets=[dbc.themes.BOOTSTRAP])
# Body
layouts=["cola", "concentric", "breadthfirst", "klay"]
app.layout = [
    #TITLE
    html.Div(
        id="title",
        children='Tool-Network'
    ),
    html.Div(
        [
            html.H2("Barra Laterale", className="display-4"),
            html.Hr(),
            html.P("Questa è una barra laterale con del testo", className="lead"),
            html.P("Puoi aggiungere altre informazioni o elementi."),
        ],
        style={
            'position': 'fixed',       # La barra resta fissa
            'top': 0,
            'left': 0,
            'bottom': 0,
            'width': '20%',            # Larghezza della barra laterale
            'padding': '20px',
            'background-color': '#f8f9fa'
        }
    ),
    
    #CLUSTER SELECTOR
    dbc.Row([
        dbc.Col(dcc.Markdown(
            id='cluster-text',
            children="Cluster selected:"
        ), width="auto"),
        dbc.Col(dcc.Dropdown(
            clusters,
            0,
            id='dropdown-cluster'
        )),
    ]),
    #FIRST ROW
    dbc.Row([
        #CYTOSCAPE BLOCK
        dbc.Col([
            #CYTOSCAPE GRAPH
            cyto.Cytoscape(
                id='cytoscape-graph',
                layout={'name': 'cola'},
                style={'width': '100%', 'height': '600px'},
                stylesheet=[
                    #NOME SOPRA
                    {
                        'selector': 'node',
                        'style': {
                            'content': 'data(name)',
                            "font-size": "5px"
                        }
                    },
                    #PAZIENTI TRIANGOLI ROSSI
                    {
                        'selector': '.PATIENT',
                        'style': {
                            'background-color': 'coral',
                            'shape': 'triangle'
                        }
                    },
                    #VARIANTI CERCHI BLUE
                    {
                        'selector': '.VARIANT',
                        'style': {
                            'background-color': 'royalblue',
                            'shape': 'circle'
                        }
                    },
                    #SELECTEDƒco
                    {
                        'selector': ':selected',
                        'style': {
                            'background-color': '#02cd79',
                        },
                    }
                ],
                elements=filter_elements_with_cluster(0),
                minZoom=0.25,
                maxZoom=2,
                responsive=True
            ),
            #INFO BOX
            dcc.Markdown(
                id='cytoscape-selectedNodeData-markdown'
            ),                    
            #LAYOUT SELECTOR
            dcc.RadioItems(
                options=layouts,
                value="cola",
                inline=True,
                id='radio-layouts'
            )
        ], lg=6),
        #FIGURE 1
        dbc.Col([
            dcc.Graph(id="fig1")
        ], lg=6)
    ]),
    #SECOND ROW
    dbc.Row([
        dbc.Row([
        #FIGURE 4
        dbc.Col([
            dcc.Graph(id="fig4")
        ], lg=14)
    ]),
   dbc.Card(
    [
        dbc.CardBody(
            html.P("Pathways Analysis", className="card-text")
        )#dbc.Button("Go somewhere", color="primary"),
            ],
    style={"width": "60%","height":"2%"},
),

        #FIGURE 2
        dbc.Col([
            dcc.RadioItems(
            options=[
                {'label': 'Biological Function', 'value': 'biological'},
                {'label': 'Molecular Function', 'value': 'molecular'},
                {'label': 'Cellular Component', 'value': 'cellular'}
            ],
            value='biological' ,
           # Valore predefinito
            labelStyle={'display': 'inline-block'},
            id="radio_fig2"
        ),html.Br(),
            dcc.Graph(id="fig2")
        ], lg=6),
                #FIGURE 3
        dbc.Col([
            dcc.Graph(id="fig3")
        ], lg=6)
    ]),
    
    
]

In [181]:
import json
with open("config.json") as f:
    data=json.load(f)
adjusted=data["Enrichment"]["adjusted"]
threshold=data["Enrichment"]["threshold"]
#SELECT NODE GRAPH
@callback(Output('cytoscape-selectedNodeData-markdown', 'children'),Input('cytoscape-graph', 'tapNodeData'))
def displaySelectedNodeData(data_dict):
    if data_dict is None:
        return "Please select node"

    if data_dict["vertex_type"] == "VARIANT":
        return f"**Variant name**: {data_dict['name']}\n**Gene**: {data_dict['gene']}\n**Sost.amm.**: {data_dict['sost_amm']}\n**Variant Classification**:{data_dict['consequence']}\n**Variant Type**:{data_dict['variant_type']}\n"
    else:
        term_excluded=["vertex_type","variants","color_vertex","shape_vertex","gene","sost_amm","variant_type","consequence","color","cluster","id","timeStamp"]
        temp=""
        for k, v in data_dict.items():
            if k not in term_excluded:
                temp+=f"**{k}**:{v}\n"
        return temp
        #return f"**Patient name**: {data_dict['name']}"

#CAMBIO LAYOUT
@callback(Output(component_id='cytoscape-graph', component_property='layout'),Input(component_id='radio-layouts', component_property='value'))
def update_graph(layout):
    return {'name': layout}

#CAMBIO CLUSTER
@callback(Output(component_id='cytoscape-graph', component_property='elements'),Input(component_id='dropdown-cluster', component_property='value'))
def update_cluster(cluster):
    return filter_elements_with_cluster(cluster)

#CAMBIO FIGURE1 GENES
@callback(Output(component_id='fig1', component_property='figure'),Input(component_id='dropdown-cluster', component_property='value'))
def update_figure1(cluster):
    df = pd.read_csv(os.path.join(output_root_path, "Gene_Count", f"genes_cluster_{cluster}.csv"),sep="\t")
    fig = px.pie(df, values='COUNT', names='GENE', title='Number Mutation for Gene')
    fig.update_traces(textposition="inside",textinfo='label')
    return fig


#CAMBIO FIGURE2 GO
@callback(Output(component_id='fig2', component_property='figure'),[Input(component_id='dropdown-cluster', component_property='value'),Input(component_id='radio_fig2', component_property='value')])
def update_figure2(cluster,process_type):
    df = pd.read_csv(os.path.join(output_root_path, "Arricchimento_all_genes", "GO", f"{process_type}_{cluster}.csv"))
    if adjusted:
        df=df[df["Adjusted.P.value"]<threshold]
        df = df.sort_values(by=['Adjusted.P.value'])[:25]
        fig=px.bar(df, x='Adjusted.P.value', y='Term',
            hover_data=['Overlap'], color='Adjusted.P.value', title='GO',labels={'Adjusted.P.value': 'Adjusted Pvalue'})
        fig.update_layout(xaxis_title="Adjusted Pvalue",  # Nome dell'asse delle x
        yaxis_title="Terms",legend_title="Adjusted Pvalue")
        return fig
    else:
        df=df[df["P.value"]<threshold]
        df = df.sort_values(by=['P.value'])[:25]
        fig=px.bar(df, x='P.value', y='Term',
            hover_data=['Overlap'], color='P.value', title='GO',labels={'P.value': 'Pvalue'})
        fig.update_layout(xaxis_title="Pvalue",  # Nome dell'asse delle x
        yaxis_title="Terms",legend_title="Pvalue")
        return fig

        

#CAMBIO FIGURE3 KEGG
@callback(Output(component_id='fig3', component_property='figure'),Input(component_id='dropdown-cluster', component_property='value'))
def update_figure3(cluster):
    df = pd.read_csv(os.path.join(output_root_path, "Arricchimento_all_genes", "KEGG", f"kegg_{cluster}.csv"))
    if adjusted:
        df=df[df["KEGG_2021_Human.Adjusted.P.value"]<threshold]
        df = df.sort_values(by=['KEGG_2021_Human.Adjusted.P.value'])[:25]
        fig= px.bar(df, x='KEGG_2021_Human.Adjusted.P.value', y='KEGG_2021_Human.Term',
            hover_data=['KEGG_2021_Human.Overlap'], color='KEGG_2021_Human.Adjusted.P.value',title='KEGG',color_continuous_scale=px.colors.sequential.Viridis,labels={'KEGG_2021_Human.P.value': 'Adjusted Pvalue'})
        fig.update_layout(xaxis_title="Adjusted Pvalue",  # Nome dell'asse delle x
        yaxis_title="Terms",legend_title="Adjusted Pvalue")
        return fig

    else:
        df=df[df["KEGG_2021_Human.P.value"]<threshold]
        df = df.sort_values(by=['KEGG_2021_Human.P.value'])[:25]
        fig= px.bar(df, x='KEGG_2021_Human.P.value', y='KEGG_2021_Human.Term',
            hover_data=['KEGG_2021_Human.Overlap'], color='KEGG_2021_Human.P.value',title='KEGG',color_continuous_scale=px.colors.sequential.Viridis,
            labels={'KEGG_2021_Human.P.value': 'Pvalue'})
        fig.update_layout(xaxis_title="Pvalue",  # Nome dell'asse delle x
        yaxis_title="Terms", legend_title="Pvalue")
        return fig

#DEGREE MUTATION 
@callback(Output(component_id='fig4', component_property='figure'),Input(component_id='dropdown-cluster', component_property='value'))
def update_figure4(cluster):
    df = pd.read_csv(os.path.join(output_root_path, "Variants_Degree",f"variants_degree_cluster{cluster}.csv"),sep="\t")
    df=df.sort_values(by=['Degree'],ascending=False)
    return px.bar(df,x="Variants",y="Degree",title="Mutation Degree")

#START
if __name__ == '__main__':
    print("Deploy on: http://127.0.0.1:8050/")
    app.run(debug=False, host='0.0.0.0')

Deploy on: http://127.0.0.1:8050/
